In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.preprocess_data import *

In [3]:
preprocess_data("../telegram_messages.json")

Preprocessed data saved to preprocessed_data.csv


In [2]:
from scripts.CoNLL_formatting import *

In [3]:
# Read the data and save the subset_df in a variable
data = 'preprocessed_data.csv'
subset_df = read_subset_df(data)

# Call the label_data function to process and save the labeled data
label_data(subset_df)

         channel         sender                  timestamp  \
0  @ZemenExpress -1001307493052  2025-03-08 13:58:44+00:00   
1  @ZemenExpress -1001307493052  2025-03-08 11:35:13+00:00   
2  @ZemenExpress -1001307493052  2025-03-08 11:34:52+00:00   
3  @ZemenExpress -1001307493052  2025-03-07 13:33:41+00:00   
4  @ZemenExpress -1001307493052  2025-03-07 13:33:40+00:00   

                                             content  \
0  💥💥...............🌞.................💥💥\n\n❓ በረፍ...   
1  💥💥....................................💥💥\n\n📌L...   
2  💥💥....................................💥💥\n\n📌L...   
3                                                NaN   
4                                                NaN   

                                              tokens  
0  ['💥💥...............🌞.................💥💥', '❓',...  
1  ['💥💥....................................💥💥', '...  
2  ['💥💥....................................💥💥', '...  
3                                                 []  
4              